In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [4]:
IMAGE_SIZE = [224,224]

Train_Path = '/content/gdrive/MyDrive/Potato/train'

In [5]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet',include_top=False)

58900480/58889256 [==============================] - 0s 0us/step


In [6]:
for layer in vgg.layers:
    layer.trainable = False

In [7]:
folder = glob('/content/gdrive/MyDrive/Apple/train/*')

In [8]:
x = Flatten()(vgg.output)

In [9]:
prediction = Dense(len(folder), activation='softmax')(x)

In [10]:
model = Model(inputs = vgg.input, outputs=prediction)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [12]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics=['accuracy']
)

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Train_data = ImageDataGenerator(
    rescale=1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

Test_data = ImageDataGenerator(rescale=1./255 )

In [14]:
Train_Dataset = Train_data.flow_from_directory('/content/gdrive/MyDrive/Potato/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1650 images belonging to 3 classes.


In [15]:
print(Train_Dataset.class_indices) #prints every single key and class of that dataset
labels = '\n'.join(sorted(Train_Dataset.class_indices.keys())) #print all these keys as a list of labels into a text file called labels.txt
with open('labels.txt', 'w') as f: #writes to the labels.txt file, and if it doesnt exists, it creates one, and if it does exist, it will overrite it. (thats what 'w' is for)
    f.write(labels)

{'Early_blight': 0, 'Healthy': 1, 'Late_blight': 2}


In [ ]:
training_set = Train_data.flow_from_directory('/content/gdrive/MyDrive/Potato/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')


test_set = Test_data.flow_from_directory('/content/gdrive/MyDrive/Potato/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1650 images belonging to 3 classes.
Found 352 images belonging to 3 classes.


In [ ]:
Test_dataset = Test_data.flow_from_directory('/content/gdrive/MyDrive/Potato/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 352 images belonging to 3 classes.


In [ ]:
p = model.fit(
  Train_Dataset,
  validation_data=Test_dataset,
  epochs=20,
  steps_per_epoch=len(Train_Dataset),
  validation_steps=len(Test_dataset)
)

Epoch 1/20


InvalidArgumentError: ignored

In [ ]:
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/20


In [ ]:
plt.plot(p.history['loss'], label='training loss')
plt.plot(p.history['val_loss'], label='validation loss')
plt.legend()
plt.show()
plt.savefig('LossValidation_loss')


plt.plot(p.history['accuracy'], label='training accuracy')
plt.plot(p.history['val_accuracy'], label='validation accuracy')
plt.legend()
plt.show()
plt.savefig('AccuracyValidation_acc')

In [ ]:
from tensorflow.keras.models import load_model

model.save('Potato_model_VGG.h5')

In [ ]:
y_pred = model.predict(Test_dataset)

In [ ]:
y_pred

In [ ]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
y_pred

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
model=load_model('Potato_model_VGG.h5')

In [ ]:
img=image.load_img('/content/gdrive/MyDrive/Apple/test/apple_rust/image (172).JPG',target_size=(224,224))

In [ ]:
x=image.img_to_array(img)
x

In [ ]:
x.shape

In [ ]:
x=x/255

In [ ]:
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

In [ ]:
model.predict(img_data)

In [ ]:
a=np.argmax(model.predict(img_data), axis=1)

In [ ]:
a == 1